# R2 Overview:

Remember to do:

Setup

List functions, then print code for one

In [193]:
import pexpect
import sys
import time

from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

rp = pexpect.spawnu('r2 -d func-pointer', encoding='utf-8')

@register_line_magic
def r(line):
    print(line)
    rp.sendline(line)
    time.sleep(.5)
    end = ""
    while not "]>" in end:
        read = rp.read_nonblocking(100000)
        print(read)
        end = read[-50:]

@register_line_magic
def ro(line):
    print(rp.read_nonblocking(100000))

@register_line_magic
def ri(line):
    rp.sendline(line)
    time.sleep(.5)
    print(rp.read_nonblocking(100000))

In [194]:
%r aa;e asm.syntax=att; afl;

aa;e asm.syntax=att; afl;
aa;e asm.syntax=att; afl;
Process with PID 4787 started...
= attach 4787 4787
bin.baddr 0x00400000
Using 0x400000
asm.bits 64
[x] Analyze all flags starting with sym. and entry0 (aa)
0x00400490    1 42           entry0
0x004004c0    4 42   -> 37   sym.deregister_tm_clones
0x004004f0    4 58   -> 55   sym.register_tm_clones
0x00400530    3 34   -> 29   entry.fini0
0x00400560    1 7            entry.init0
0x00400660    1 2            sym.__libc_csu_fini
0x00400664    1 9            sym._fini
0x00400593    1 22           sym.other
0x00400470    1 6            sym.imp.printf
0x004005f0    4 101          sym.__libc_csu_init
0x00400567    1 27           sym.special
0x00400460    1 6            sym.imp.puts
0x004005a9    1 58           main
0x00400480    1 6            sym.imp.gets
0x00400582    1 17           sym.normal
0x00400438    3 23           sym._init
[0x7f5ba8dc7090]> 


In [195]:
%r pdf @main

pdf @main
            ; DATA XREF from entry0 @ 0x4004ad
┌ 58: int main (int argc, char **argv, char **envp);
│           ; var int64_t var_30h @ rbp-0x30
│           ; var int64_t var_24h @ rbp-0x24
│           ; var int64_t var_16h @ rbp-0x16
│           ; var int64_t var_8h @ rbp-0x8
│           ; arg int argc @ rdi
│           ; arg char **argv @ rsi
│           0x004005a9      55             pushq %rbp
│           0x004005aa      4889e5         movq %rsp, %rbp
│           0x004005ad      4883ec30       subq $0x30, %rsp
│           0x004005b1      897ddc         movl %edi, var_24h          ; argc
│           0x004005b4      488975d0       movq %rsi, var_30h          ; argv
│           0x004005b8      48c745f88205.  movq $sym.normal, var_8h    ; 0x400582
│           0x004005c0      488d45ea       leaq var_16h, %rax
│           0x004005c4      4889c7         movq %rax, %rdi
│           0x004005c7      b800000000     movl $0, %eax
│           0x004005cc      e8affeffff     callq sym.i

# R2: Set Break Points:


In [196]:
%r db 0x004005d1

db 0x004005d1
[0x7f5ba8dc7090]> K


In [197]:
%r dc

dc
[0x7f5ba8dc7090]> dc


# R2 Interact:

In [198]:
my_str = "aaaaaaaaaaaaaa"+chr(int("67",16))+chr(int("05",16))+chr(int("40",16))+chr(int("00",16))
print(my_str)
len(my_str)

aaaaaaaaaaaaaag@ 


18

In [199]:
%ri {my_str}

aaaaaaaaaaaaaag^E@^@
hit breakpoint at: 4005d1
[0x004005d1]> 


In [200]:
%r pdf

pdf
            ; DATA XREF from entry0 @ 0x4004ad
┌ 58: int main (int argc, char **argv, char **envp);
│           ; var int64_t var_30h @ rbp-0x30
│           ; var int64_t var_24h @ rbp-0x24
│           ; var int64_t var_16h @ rbp-0x16
│           ; var int64_t var_8h @ rbp-0x8
│           ; arg int argc @ rdi
│           ; arg char **argv @ rsi
│           0x004005a9      55             pushq %rbp
│           0x004005aa      4889e5         movq %rsp, %rbp
│           0x004005ad      4883ec30       subq $0x30, %rsp
│           0x004005b1      897ddc         movl %edi, var_24h          ; argc
│           0x004005b4      488975d0       movq %rsi, var_30h          ; argv
│           0x004005b8      48c745f88205.  movq $sym.normal, var_8h    ; 0x400582
│           0x004005c0      488d45ea       leaq var_16h, %rax
│           0x004005c4      4889c7         movq %rax, %rdi
│           0x004005c7      b800000000     movl $0, %eax
│           0x004005cc      e8affeffff     callq sym.imp.get

# Find Target:

In [201]:
%r pdf @sym.normal

pdf @sym.normal
            ; DATA XREF from main @ 0x4005b8
┌ 17: sym.normal ();
│           0x00400582      55             pushq %rbp
│           0x00400583      4889e5         movq %rsp, %rbp
│           0x00400586      bfb3064000     movl $str.this_is_the_normal_function, %edi ; 0x4006b3 ; "this is the normal function"
│           0x0040058b      e8d0feffff     callq sym.imp.puts          ; int puts(const char *s)
│           0x00400590      90             nop
│           0x00400591      5d             popq %rbp
└           0x00400592      c3             retq
[0x004005d1]> 


In [202]:
%r pdf @sym.special

pdf @sym.special
┌ 27: sym.special ();
│           0x00400567      55             pushq %rbp
│           0x00400568      4889e5         movq %rsp, %rbp
│           0x0040056b      bf80064000     movl $str.this_is_the_special_function, %edi ; 0x400680 ; "this is the special function"
│           0x00400570      e8ebfeffff     callq sym.imp.puts          ; int puts(const char *s)
│           0x00400575      bf9d064000     movl $str.you_did_this__friend, %edi ; 0x40069d ; "you did this, friend!"
│           0x0040057a      e8e1feffff     callq sym.imp.puts          ; int puts(const char *s)
│           0x0040057f      90             nop
│           0x00400580      5d             popq %rbp
└           0x00400581      c3             retq
[0x004005d1]> 


# Inspect Variables:

In [203]:
%r px 20 @ rbp-0x16

px 20 @ rbp-0x16
- offset -       0 1  2 3  4 5  6 7  8 9  A B  C D  E F  0123456789ABCDEF
0x7ffda66baaca  6161 6161 6161 6161 6161 6161 6161 6705  aaaaaaaaaaaaaag.
0x7ffda66baada  4000 0000                                @...
[0x004005d1]> 


In [204]:
# Need to overwrite this:
%r px 16 @ rbp-0x8

px 16 @ rbp-0x8
- offset -       0 1  2 3  4 5  6 7  8 9  A B  C D  E F  0123456789ABCDEF
0x7ffda66baad8  6705 4000 0000 0000 f005 4000 0000 0000  g.@.......@.....
[0x004005d1]> 


In [35]:
%r px 16 @ rdx

- offset -   0 1  2 3  4 5  6 7  8 9  A B  C D  E F  0123456789ABCDEF
0x00400500  48c1 fe03 4889 f048 c1e8 3f48 01c6 48d1  H...H..H..?H..H.
[0x004005da]> 


In [22]:
%r dr

rax = 0x7ffd7c2f392a0m dr
rbx = 0x00000000
rcx = 0x7f60d3f53980
rdx = 0x00000000
r8 = 0x7ffd7c2f392a
r9 = 0x00000000
r10 = 0x0000006e
r11 = 0x00000246
r12 = 0x00400490
r13 = 0x00000000
r14 = 0x00000000
r15 = 0x00000000
rsi = 0x6767676767676767
rdi = 0x7f60d3f56680
rsp = 0x7ffd7c2f3910
rbp = 0x7ffd7c2f3940
rip = 0x004005d1
rflags = 0x00000206
orax = 0xffffffffffffffff
[0x004005d1]> 


# Step Through Program:

In [36]:
%r ds; pdf;

            ; CODE XREF from entry.init0 @ 0x400565
┌ 55: sym.register_tm_clones ();
│           0x004004f0      be38106000     movl $loc._end, %esi        ; obj.__TMC_END
│                                                                      ; 0x601038
│           0x004004f5      55             pushq %rbp
│           0x004004f6      4881ee381060.  subq $loc._end, %rsi        ; obj.__TMC_END
│                                                                      ; 0x601038
│           0x004004fd      4889e5         movq %rsp, %rbp
│           ;-- rdx:
│           0x00400500      48c1fe03       sarq $3, %rsi
│           0x00400504      4889f0         movq %rsi, %rax
│           0x00400507      48c1e83f       shrq $0x3f, %rax
│           0x0040050b      4801c6         addq %rax, %rsi
│           0x0040050e      48d1fe         sarq $1, %rsi
│       ┌─< 0x00400511      7415           je 0x400528
│       │   0x00400513      b800000000     movl $0, %eax
│       │   0x00400518      4885c0     

In [205]:
%r dc

dc
this is the special functionK
you did this, friend!
[0x7f5ba8cb3659]> 


# Other Notes: